In [2]:
pip install exif tqdm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 70.0/70.0 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
'''READ ME:

This is a Python Jupyter Notebook desinged to run the location, time, and elevation extraction of drone images for projects. The output is saved to a
csv which can be added to an experience builder application. Run the pip install exif if errors come up and restart the 
kernel as needed.

Note, if the csv file already exists, the tool will fail. Please make sure to close any files being called to write on 
before running.

Sebastian Brauer - 2024/06/11
'''

image_folder = r'P:\Data\Drone Videos\MnDOT-D4\Dillworth_Moorhead\Existing\Drone Images' # REQUIRED USER INPUT needed to know where images are
output_csv = r'C:\Users\sbrauer\Downloads\Dillworth_Images.csv' # REQUIRED USER INPUT output save location
input_sharepoint_link = 'https://isthmuseng.sharepoint.com/sites/000-018/Shared%20Documents/Drone%20Images/Dillworth/Drone%20Images/' # REQUIRED USER INPUT location on sharepoint where the images are stored

import os
import csv
from exif import Image
from tqdm import tqdm

# Function to get EXIF data from an image
def get_exif_data(image_path):
    with open(image_path, 'rb') as image_file:
        image = Image(image_file)
        if image.has_exif:
            return {
                'file_name': os.path.basename(image_path),
                'latitude': getattr(image, 'gps_latitude', None),
                'latitude_ref': getattr(image, 'gps_latitude_ref', None),
                'longitude': getattr(image, 'gps_longitude', None),
                'longitude_ref': getattr(image, 'gps_longitude_ref', None),
                'capture_time': getattr(image, 'datetime_original', None),
                'elevation': getattr(image, 'gps_altitude', None)
            }
        else:
            return None

# Convert GPS coordinates to decimal degrees
def convert_to_degrees(gps_coord, ref):
    if gps_coord:
        d, m, s = gps_coord
        degrees = d + (m / 60.0) + (s / 3600.0)
        if ref in ['S', 'W']:
            degrees = -degrees
        return degrees
    return None

# Create CSV file with header
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image Number', 'Latitude', 'Longitude', 'Image Capture Time', 'Elevation', 'Image Link'])
    
    image_names = os.listdir(image_folder)

    # Iterate through images in the folder
    for idx, image_name in enumerate(tqdm(image_names, desc="Processing Images")):
        image_path = os.path.join(image_folder, image_name)
        exif_data = get_exif_data(image_path)
        image_link = input_sharepoint_link + image_name

        if exif_data:
            latitude = convert_to_degrees(exif_data['latitude'], exif_data['latitude_ref'])
            longitude = convert_to_degrees(exif_data['longitude'], exif_data['longitude_ref'])
            capture_time = exif_data['capture_time']
            elevation = exif_data['elevation']

            # Write data to CSV
            writer.writerow([image_name, latitude, longitude, capture_time, elevation, image_link])

'''
If you get the following error: ModuleNotFoundError: No module named 'exif'. Then run the cell below first by clicking the run button again. Then click the restart button next the run button.
'''
            
print(f"CSV file has been created at {output_csv}")

Processing Images: 100%|███████████████████████████████████████████████████████████████| 49/49 [00:13<00:00,  3.57it/s]

CSV file has been created at C:\Users\sbrauer\Downloads\Dillworth_Images.csv
